In [25]:
import openmc
from openmc.deplete import Chain
from pandas import DataFrame
from openmc.mgxs import FissionXS, ArbitraryXS, EnergyGroups

In [31]:
model = openmc.Model.from_xml()
chain_file = '../openmc/tests/chain_simple.xml'
chain = Chain.from_xml(chain_file)
reactions = chain.reactions

groups = EnergyGroups((0,20e6))
reaction_domain=model.materials[0]
xs = {}
for rx in reactions:
    if rx == 'fission':
        xs[rx] = FissionXS(domain=reaction_domain,
                     energy_groups=groups, by_nuclide=True)
    else:
        xs[rx] = ArbitraryXS(rx, domain=reaction_domain,
                       energy_groups=groups, by_nuclide=True)
        
sps = []
for i in range(0,7):
    sps.append(openmc.StatePoint(f'openmc_simulation_n{i}.h5'))
    tally = sps[i].get_tally(scores=['flux', 'heating'])
    flux = tally.get_slice(scores=['flux'])
    heating = tally.get_slice(scores=['heating'])
    phi = flux.mean
    H = heating.mean

    H *= openmc.data.JOULE_PER_EV
    vol = 1.24**2
    f = 174 / H
    Phi = f * phi / vol
    print(f'Step: {i}')
    print(f'Flux: {Phi}')


dfs = []
for sp in sps:
    for rx in xs:
        xs[rx].load_from_statepoint(sp)
    sp.close()
    series = {}
    for rx in xs:
        df = xs[rx].get_pandas_dataframe(xs_type='micro')
        series[rx] = df.set_index('nuclide')['mean']
    dfs.append(DataFrame(series))


/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=1.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=2.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=3.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=4.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: ID

Step: 0
Flux: [[[4.75093051e+14]]]
Step: 1
Flux: [[[4.73053168e+14]]]
Step: 2
Flux: [[[4.76126636e+14]]]
Step: 3
Flux: [[[4.74588439e+14]]]
Step: 4
Flux: [[[4.76227708e+14]]]
Step: 5
Flux: [[[4.74706037e+14]]]
Step: 6
Flux: [[[4.74930105e+14]]]


In [32]:
dfs

[         (n,gamma)    fission
 nuclide                      
 U234      9.438788   0.214529
 U235      4.545681  21.000721
 U238      0.376799   0.045651
 O16       0.000032   0.000000
 O17       0.000177   0.000000
 U236      3.735857   0.138560,
          (n,gamma)    fission
 nuclide                      
 U234      9.351518   0.215381
 U235      4.554530  21.040166
 U238      0.378707   0.045929
 O16       0.000033   0.000000
 O17       0.000177   0.000000
 U236      3.997183   0.142015,
          (n,gamma)    fission
 nuclide                      
 U234      9.722396   0.216330
 U235      4.550834  21.016055
 U238      0.377679   0.046213
 O16       0.000033   0.000000
 O17       0.000177   0.000000
 U236      3.727184   0.140099,
          (n,gamma)    fission
 nuclide                      
 U234      9.314502   0.216668
 U235      4.521879  20.910415
 U238      0.374590   0.046532
 O16       0.000033   0.000000
 O17       0.000176   0.000000
 U236      3.889316   0.142208,
    

In [17]:
tallies[2].

Tally
	ID             =	3
	Name           =	
	Filters        =	UniverseFilter, EnergyFilter
	Nuclides       =	U234 U235 U238 U236 O16 O17 Zr90 Zr91 Zr92 Zr94 Zr96 H1 H2
	Scores         =	['(n,gamma)']
	Estimator      =	tracklength